In [ ]:
import tensorflow as tf
import importlib
import os
import pickle
from models import *
from dataprocessor import *
from tqdm import tqdm
from change_point_detection import *
def reload_custom_libs():
    import loss_functions
    import models
    import dataprocessor
    importlib.reload(loss_functions)
    importlib.reload(models)
    importlib.reload(dataprocessor)

In [4]:
# 加载文件
macd_timescales = [(8, 24), (16, 28), (32, 96)]
rtn_timescales = [1, 21, 63, 126, 252]
timesteps = 126

folder_path = "data"
files = [f for f in os.listdir(folder_path) if f.endswith(".xlsx")]
files = [
    file
    for file in files
    if file
    not in (
        "CC00.NYB.xlsx",
        "LB00.CME.xlsx",
        "ES00.CME.xlsx",
        "NQ00.CME.xlsx",
        "YM00.CBT.xlsx",
        "SP00.CME.xlsx",
    )
]
data_list = []


data_list = process_data_list(files, macd_timescales, rtn_timescales, test=True)

处理文件中。。: 100%|██████████| 11/11 [00:58<00:00,  5.29s/it]


In [ ]:
# 试试全部的数据。。

tmp = []
for data in tqdm(data_list):
    price_series = data["close"]
    target = price_series.to_numpy().reshape((-1, 1))
    segment_list = get_segment_points(target, l_max=63, mu=0.999)
    segment_list = [data.iloc[start : end, :] for start, end in segment_list]
    tmp.extend(segment_list)

In [5]:
# 跑过一次的话，不要再跑入这个

gaussion_process_list = []
for data in tqdm(data_list):
    price_series = data["close"]
    target = price_series.to_numpy().reshape((-1, 1))
    segment_list = get_segment_points(target, l_max=63, mu=0.999)
    segment_list = [data.iloc[start : end, :] for start, end in segment_list]
    gaussion_process_list.extend(segment_list)

100%|██████████| 11/11 [29:30<00:00, 160.98s/it]


In [215]:
# 生成 taget 信息
reload_custom_libs()
from dataprocessor import *
from models import *
target_set, labels, map = generate_tensors(data_list, time_steps=timesteps, encoder_type = "one-hot", return_map=True)


生成张量, 并对类别信息进行one-hot 编码: 100%|██████████| 11/11 [00:00<00:00, 25.10it/s]

one-hot 编码中...


In [211]:
feature_cols = gaussion_process_list[0].columns.to_list()
for col in ("date", "sigma", "side_info", "close", "rtn"):
    feature_cols.remove(col)

features = [df[feature_cols].values for df in gaussion_process_list if len(df) > 0]
date_list = [str(pd.to_datetime(df["date"]).dt.strftime("%Y-%m-%d").values[-1]) for df in gaussion_process_list if len(df) > 0]
side_info = [df["side_info"].iloc[0] for df in gaussion_process_list if len(df) > 0]
rtn_next_day = [df["rtn_next_day"] for df in gaussion_process_list if len(df) > 0]
std = [df["sigma"] for df in gaussion_process_list if len(df) > 0]

feature_sequences_tensor = tf.ragged.constant(features, dtype=tf.float32)
dates_tensor = tf.constant(date_list, dtype=tf.string)
side_info_tensor = tf.constant(side_info, dtype=tf.string)
rtn_next_day = tf.ragged.constant(rtn_next_day)
std = tf.ragged.constant(std)

def pad_ragged_tensors(tensor):
    padded = tensor.to_tensor(0.)
    return tf.reverse(padded, [-1])

feature_sequences_tensor = pad_ragged_tensors(feature_sequences_tensor)
rtn_next_day = pad_ragged_tensors(rtn_next_day)
std = pad_ragged_tensors(std)

side_info_tensor, map = side_info_one_hot_encoder(side_info_tensor, time_steps=feature_sequences_tensor.shape[1], word_index=map, return_dict=True)
context_set = (feature_sequences_tensor, dates_tensor, side_info_tensor)

In [221]:
import tensorflow as tf

bind_number = 2 # 实际中应该是总资产数目
features = target_set[0]
dates = target_set[1]
context = target_set[2]
rtn = labels[0]
std = labels[1]

unique_dates, _, counts = tf.unique_with_counts(dates)
twice_dates = tf.boolean_mask(unique_dates, counts == 2)

# Step 2: Create a mapping from dates to indices
date_to_indices = {date.numpy(): tf.where(dates == date).numpy().flatten() for date in twice_dates}

# Step 3: Gather features and contexts for these dates, ensuring the order is maintained
ordered_features = []
ordered_contexts = []
ordered_rtn, orderd_std = [], []

for date in twice_dates:
    indices = date_to_indices[date.numpy()]
    ordered_features.append(tf.gather(features, indices))
    ordered_contexts.append(tf.gather(context, indices))
    ordered_rtn.append(tf.gather(rtn, indices))
    orderd_std.append(tf.gather(std, indices))

ordered_features = tf.stack(ordered_features)
ordered_contexts = tf.stack(ordered_contexts)
taget_set = (ordered_features, twice_dates, ordered_contexts)
labels = (ordered_rtn, orderd_std)

In [199]:
# 把 context 绑定
bind_number = 2 # 实际中应该是目标context的数目

feature_sequences_tensor = context_set[0]
dates_tensor = context_set[1]
dates_tensor_int = [date_to_int(date) for date in dates_tensor.numpy()]
side_info_tensor = context_set[2]

unique_elements, idx, counts = tf.unique_with_counts(dates_tensor)

new_feature_sequences = []
new_side_info_sequences = []
binded_dates = []
for date in dates.numpy():
    # 找到比当前日期小的日期
    smaller_dates_indices = tf.where(dates_tensor_int < date_to_int(date))
    if len(smaller_dates_indices) > 0:
        # 选择一个随机的日期
        shuffled_indices = tf.random.shuffle(smaller_dates_indices)
        chosen_index = shuffled_indices[0, 0]
        current_date_indices = tf.where(dates_tensor == date)
        
        for idx in current_date_indices:
            # 提取对应的 feature 和 side_info
            new_feature_sequences.append(tf.stack([feature_sequences_tensor[chosen_index], feature_sequences_tensor[idx[0]]]))
            new_side_info_sequences.append(tf.stack([side_info_tensor[chosen_index], side_info_tensor[idx[0]]]))
            binded_dates.append(date)

binded_feature_sequences_tensor = tf.stack(new_feature_sequences)
binded_side_info_sequences_tensor = tf.stack(new_side_info_sequences)
binded_dates_tensor = tf.constant(binded_dates)

# 输出结果
print("New feature sequences shape:", binded_feature_sequences_tensor.shape)
print("New side info sequences shape:", binded_side_info_sequences_tensor.shape)
print("New dates shape:", binded_dates_tensor.shape)

context_set = (binded_feature_sequences_tensor, binded_dates_tensor, binded_side_info_sequences_tensor)

New feature sequences shape: (99, 2, 71, 9)
New side info sequences shape: (99, 2, 71, 12)
New dates shape: (99,)


In [224]:
context_dates_set = set(context_set[1].numpy())

# 筛选 target_dates 中属于 context_dates 的索引
indices = [i for i, date in enumerate(target_set[1].numpy()) if date in context_dates_set]

In [185]:
context_dates_set = set(context_set[1].numpy())

# 筛选 target_dates 中属于 context_dates 的索引
indices = [i for i, date in enumerate(target_set[1].numpy()) if date in context_dates_set]

# 根据索引筛选 target_set
filtered_target_feature_sequences = tf.gather(target_set[0], indices)
filtered_target_dates = tf.gather(target_dates, indices)
filtered_target_side_info = tf.gather(target_side_info, indices)

((<tf.Tensor: shape=(376, 2, 126, 8), dtype=float32, numpy=
  array([[[[-0.66853815, -1.0418897 , -1.4385751 , ..., -0.8337918 ,
             0.71539265,  2.5282018 ],
           [-0.5658393 , -0.95480883, -1.375749  , ..., -0.6738516 ,
             0.661643  ,  2.2054584 ],
           [-0.48268455, -0.8775273 , -1.3158532 , ..., -1.1159526 ,
             0.36439225,  2.4865031 ],
           ...,
           [-1.4719111 , -1.3283683 , -1.0933195 , ..., -0.32118016,
            -0.7874642 , -0.46159858],
           [-1.5025609 , -1.3742076 , -1.160879  , ..., -0.31145963,
            -0.833678  , -0.48138905],
           [-1.5332779 , -1.416074  , -1.222581  , ..., -0.74657243,
            -0.9569377 , -0.57225454]],
  
          [[-0.5658393 , -0.95480883, -1.375749  , ..., -0.6738516 ,
             0.661643  ,  2.2054584 ],
           [-0.48268455, -0.8775273 , -1.3158532 , ..., -1.1159526 ,
             0.36439225,  2.4865031 ],
           [-0.4768266 , -0.84639   , -1.2803382 , ..., 

# 模型预测

In [121]:
import models
importlib.reload(models)
from dataprocessor import *
from models import *

batch_size = len(files)
hidden_dim = 64 # 128
warm_up = 63
target_std = tf.cast(5e-2, tf.float64)
features_len = len(macd_timescales) + len(rtn_timescales)
encoding_size = len(files) + 1
x_shape = (batch_size, timesteps, features_len)
s_shape = (batch_size, timesteps, encoding_size)

model = ModelWrapper(features_len, hidden_dim, encoding_size, num_heads = 4)
model.build((x_shape, s_shape))

x_c, x_c_rtn, s_c = feature_sequences_tensor[:, :, 1:], feature_sequences_tensor, side_info_tensor
x, s = target_set[0], target_set[-1]
result = model(x_c, x_c_rtn, s_c, x, s)
